# Notebook to test the completeness check functionality of the IBJ project.

Test on Burundi, 3rd country most visited and critical on human rights issues.

In [2]:
# LIBRARIES ---------------------------------------------------
import os
os.chdir('/Users/dianaavalos/PycharmProjects/InternationalBridgesToJustice')
import json
from pprint import pprint
from src.internationalbridgestojustice.config import Paths
from src.internationalbridgestojustice.openai_utils import openai_client
from src.internationalbridgestojustice.chromadb_utils import load_collection
from src.internationalbridgestojustice.get_completeness import KeypointEvaluation
from src.internationalbridgestojustice.file_manager import get_country_names

Load the prompt and system prompt for the completeness check.

In [3]:
with open(Paths.PATH_FILE_PROMPT_COMPLETENESS, "r") as file:
    prompt_completeness = file.read()

with open(Paths.PATH_FILE_SYSTEM_PROMPT_COMPLETENESS, "r") as file:
    system_prompt = file.read()

print(prompt_completeness[:1000])

In [4]:
print(system_prompt)

You are a critical legal analyst tasked with evaluating whether a legal wiki chapter adequately addresses a specific legal keypoint. 
Your response must be precise, structured, and based on legal reasoning. 
When relevant, cite and summarize laws from the provided legal database. 
Avoid vague language and clearly distinguish between complete, partial, or missing legal coverage.



In [5]:
with open(Paths.PATH_JSON_FILE_DESCRIPTIVE_COMPLETENESS_KEYPOINTS, "r") as file:
        completeness_keypoints = json.load(file)    
pprint(completeness_keypoints[5:8])

[{'Description': 'This section provides an overview of the legal aid landscape '
                 'within the country, specifically detailing the total number '
                 'of practicing lawyers categorized by their specialization in '
                 'criminal and civil law, if such data is available.',
  'Keypoint': '1. Introduction: 3. The legal aid situation in the country: 3. '
              'Number of lawyers (criminal/civil) if known'},
 {'Description': 'This section aims to explore and identify the various legal '
                 'foundations and sources that establish and protect the '
                 'rights of defendants in a legal context.',
  'Keypoint': '1. Introduction: 4. Sources of defendant’s rights?'},
 {'Description': "This section introduces the topic of defendant's rights and "
                 'specifically focuses on the national legal frameworks and '
                 'constitutional provisions that establish and protect these '
                 'right

In [6]:
collection = load_collection(Paths.PATH_CHROMADB, Paths.COLLECTION_NAME)

/Users/dianaavalos/PycharmProjects/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Print the keypoints to check for completeness.

In [7]:
country_names = get_country_names(country_names_filepath="data/interim/country_names_1.txt")
country = "Burundi"
keypoint = completeness_keypoints[10]
evaluation = KeypointEvaluation(country=country, keypoint=keypoint, system_prompt=system_prompt, model="gpt-4o-mini", collection=collection, lazy=True)
evaluation

<KeypointEvaluation(Burundi, 2. Rights of the Accused: 2. Right Against Unlawful Detention...)>

In [8]:
evaluation.run_similarity_searches(collection=collection)
evaluation.define_prompt(prompt_template=prompt_completeness)
print(evaluation.prompt [:1000])

## **Completeness Instructions**

You are tasked with evaluating whether the following wiki chapter sufficiently and accurately addresses the key point: **2. Rights of the Accused: 2. Right Against Unlawful Detention** , with is best describe as: **The accused individuals have the fundamental right to be free from detention that is not legally justified or authorized, ensuring their protection against arbitrary imprisonment.**.

Before making your judgment, critically analyze **both** the wiki content and the legal database. Even if the chapter appears thorough, investigate whether specific legal provisions could **strengthen, clarify, or correct** the information. Your assessment should prioritize **clarity, specificity**, and **legal accuracy**.

---

### **Classification Categories**

Choose **one** of the following categories:

* **Complete**:
  The chapter clearly addresses the key point with sufficient legal detail, citing **specific laws or articles**, and outlining relevant **r

In [9]:
evaluation.check_completeness(client=openai_client, temperature=0.1)

In [10]:
# evaluation.answer = json.loads(evaluation.answer) #loads the str into a dict # done in check_completeness now
pprint(evaluation.answer)

{'Classification': 'Needs Refinement',
 'Country': 'Burundi',
 'Keypoint': '2. Right Against Unlawful Detention',
 'Keypoint_Description': 'The accused individuals have the fundamental right '
                         'to be free from detention that is not legally '
                         'justified or authorized, ensuring their protection '
                         'against arbitrary imprisonment.',
 'Legal_Provisions_Check': 'Present',
 'Missing_or_Unclear': 'The chapter does not explicitly address the right '
                       'against unlawful detention as a standalone right. '
                       'While it mentions related rights, it lacks a clear '
                       'definition and specific legal references that directly '
                       'support the right against unlawful detention. The '
                       'relevant legal provisions from the database are not '
                       'cited, which would strengthen the legal grounding of '
             

Save answer and log info

In [11]:
evaluation.save_answer_as_json()
evaluation.save_log_as_json()

## Now we want to do exactly this in batches per country with all keypoints.

In [13]:
request = evaluation.build_batch_request(
    custom_id=f"{country}-{keypoint}",
    user_prompt=evaluation.prompt,
    temperature=0.1
)

In [14]:
jsonl_file_completeness_batch = f"{Paths.PATH_FOLDER_COMPLETENESS}/batch_input_1.jsonl"
with open(jsonl_file_completeness_batch, "a") as outfile:
    outfile.write(json.dumps(request) + "\n")
